# **Kỹ thuật tensorflow Dataset**

## **1. Vai trò của tensorflow Dataset**

**Vì sao có thể truyền các bộ dữ liệu lớn vào mô hình?**

<p>Ta không thể tuyền toàn bộ dữ liệu vào mô hình cùng một lúc vì dữ liệu thường có kích thước lớn hơn RAM máy tính. Các framework deep learning thường hỗ trợ huấn luyện mô hình theo generator. Dữ liệu sẽ được load theo từng batch, tức huấn luyện đến đâu thì khởi tạo đến đó.</p>

<p>Tùy theo định dạng dữ liệu là text, image, dataframe, numpy array,... mà chúng ta sẽ sử dụng những module tạo dữ liệu huấn luyện khác nhau.</p>


## **2. Định nghĩa generator**

Generator có thể coi là một người vay nợ, được quyền sử dụng của người khác nhưng không trả ngay, ta ví dữ liệu như là tiền thì generator sẽ sử dụng dữ liệu vào mục đích của mình. Tuy nhiên dữ liệu sau khi biến đổi không được trả về như các hàm thông thường.

Giả sử một người vay $n$ món nợ với cùng lãi suất là 1%/tháng. Để tính lãi suất phải trả của các khoản vay chúng ta có thể sử dụng vòng for và tính để tính kết quả trong 1 lần.

In [1]:
import numpy as np
from datetime import datetime

def _interest_rate(month):
  return (1+0.01)**month - 1


periods = [1, 3, 6, 9, 12]
scales = [_interest_rate(month) for month in periods]
print('scales of origin balance: ', scales)

scales of origin balance:  [0.010000000000000009, 0.030301000000000133, 0.061520150601000134, 0.09368527268436089, 0.12682503013196977]


Tuy nhiên nếu sử dụng generator thì chúng ta chỉ việc thay `return` bằng `yield`.

In [2]:
def _gen_interest_rate(month):
  yield (1+0.01)**month - 1


periods = [1, 3, 6, 9, 12]
scales = [_gen_interest_rate(month) for month in periods]
print('scales of origin balance: ', scales)

scales of origin balance:  [<generator object _gen_interest_rate at 0x0000021F7CFB4DD0>, <generator object _gen_interest_rate at 0x0000021F7CFB4430>, <generator object _gen_interest_rate at 0x0000021F7CFB44A0>, <generator object _gen_interest_rate at 0x0000021F7CFB4660>, <generator object _gen_interest_rate at 0x0000021F7CFB4580>]


generator sẽ không trả về kết quả ngay mà chỉ tạo sẵn các ô nhớ lưu hàm generator mô tả cách tính lãi suất. Không tốn chi phí thời gian thực hiện phép tính -> đang nợ máy tính kết quả trả về. Chỉ khi nào được chủ nợ gọi tên bằng cách kích hoạt trong ham `next()` thì mới tính kết quả.

In [3]:
[next(_gen_interest_rate(n)) for n in periods]

[0.010000000000000009,
 0.030301000000000133,
 0.061520150601000134,
 0.09368527268436089,
 0.12682503013196977]

Lợi thế generator là:
- Không sinh toàn bộ dữ liệu cùng một lúc, nâng cao hiệu suất vì sử dụng ít bộ nhớ hơn.
- Không phải chờ toàn bộ các vòng lặp được xử lý xong thì mới xử lý tiếp nên tiết kiệm thời gian tính toán 

## **3. Cách khởi tạo một Dataset:**

Dataset là một class của tensorflow dùng để wrap dữ liệu trước khi truyền vào mô hình để huấn luyện để kiểm soát input X, output y. Ta nên wrap chúng trong `tf.Dataset`

Có 2 phương pháp chính để khởi tạo một tf.Dataset trong tensorflow:

* **In memory Dataset:** Khởi tạo các dataset ngay từ đầu và dữ liệu được lưu trữ trên memory.
* **Generator Dataset:** Dữ liệu được sinh ra theo từng batch và xen kẽ với quá trình huấn luyện từ các hàm khởi tạo generator.


Phương pháp `In memory Dataset` sẽ phù hợp với các bộ dữ liệu kích thước nhỏ mà RAM có thể load được. Quá trình huấn luyện theo cách này thì nhanh hơn so với phương pháp `Generator Dataset` vì dữ liệu đã được chuẩn bị sẵn mà không tốn thời gian chờ khởi tạo batch. Tuy nhiên dễ xảy ra `out of memory` trong quá trình huấn luyện.

Theo cách `Generator Dataset` chúng ta sẽ qui định cách mà dữ liệu được tạo ra như thế nào thông qua một hàm `generator`. Quá trình huấn luyện đến đâu sẽ tạo batch đến đó. Do đó các bộ dữ liệu big data có thể được load theo từng batch sao cho kích thước vừa được dung lượng RAM. Theo cách huấn luyện này chúng ta có thể huấn luyện được các bộ dữ liệu có kích thước lớn hơn nhiều so với RAM bằng cách chia nhỏ chúng theo batch. Đồng thời có thể áp dụng thêm các step preprocessing data trước khi dữ liệu được đưa vào huấn luyện. Do đó đây thường là phương pháp được ưa chuộng khi huấn luyện các model deep learning.

### **3.1 In Memory Dataset**

In [4]:
from tensorflow.keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(60000, 28, 28)
(10000, 28, 28)
(60000,)
(10000,)


Các dữ liệu train và test được load vào bộ nhớ. Tiếp theo chúng ta sẽ khởi tạo Dataset cho những dữ liệu in memory này bằng hàm `tf.data.Dataset.from_tensor_slices()`

In [5]:
import tensorflow as tf
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
valid_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

Khi đó chúng ta đã có thể fit vào mô hình huấn luyện các dữ liệu được truyền vào `tf.Dataset` là `(X_train, y_train)`.

Chúng ta cũng có thể áp dụng các phép biến đổi bằng các hàm như `Dataset.map()` hoặc `Dataset.batch()` để biến đổi dữ liệu trước khi fit vào model. Các bạn xem thêm tại `tf.Dataset`. Chẳng hạn trước khi truyền batch vào huấn luyện tôi sẽ thực hiện chuẩn hóa batch theo phân phối chuẩn.

In [6]:
import numpy as np
from tensorflow.keras.backend import std, mean
from tensorflow.math import reduce_std, reduce_mean

def _normalize(X_batch, y_batch):
  '''
  X_batch: matrix digit images, shape batch_size x 28 x 28
  y_batch: labels of digit.
  '''
  X_batch = tf.cast(X_batch, dtype = tf.float32)
  # Padding về 2 chiều các giá trị 0 để được shape là 32 x 32
  pad = tf.constant([[0, 0], [2, 2], [2, 2]])
  X_batch = tf.pad(X_batch, paddings=pad, mode='CONSTANT', constant_values=0)
  X_batch = tf.expand_dims(X_batch, axis=-1)
  mean = reduce_mean(X_batch)
  std = reduce_std(X_batch)
  X_norm = (X_batch-mean)/std
  return X_norm, y_batch

train_dataset = train_dataset.batch(32).map(_normalize)
valid_dataset = valid_dataset.batch(32).map(_normalize)

`train_dataset` và `valid_dataset` lần lượt thực hiện các bước xử lí dữ liệu sau:
- Hàm `.batch(32)`: Trích xuất ra từ list `(X_train, y_train)` các batch_size có kích thước là 32.
- Hàm `.map(_normalize)`: Mapping đầu vào là các batch `(X_batch, y_batch)` kích thước 32 vào hàm số `_normalize()`. Kết quả trả về là giá trị đã chuẩn hóa theo batch của `x_batch` và `y_batch`. Dữ liệu này sẽ được sử dụng để huấn luyện model.

In [7]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

base_extractor = MobileNet(input_shape = (32, 32, 1), include_top = False, weights = None)
flat = Flatten()
den = Dense(10, activation='softmax')
model = Sequential([base_extractor, 
                   flat,
                   den])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_32 (Function  (None, 1, 1, 1024)       3228288   
 al)                                                             
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 10)                10250     
                                                                 
Total params: 3,238,538
Trainable params: 3,216,650
Non-trainable params: 21,888
_________________________________________________________________


In [8]:
model.compile(Adam(), loss='sparse_categorical_crossentropy', metrics = ['accuracy'])
model.fit(train_dataset, validation_data= valid_dataset,epochs = 2)

Epoch 1/2
1875/1875 [==============================] - 485s 257ms/step - loss: 0.4860 - accuracy: 0.8490 - val_loss: 0.1739 - val_accuracy: 0.9488
Epoch 2/2
1875/1875 [==============================] - 463s 247ms/step - loss: 0.1550 - accuracy: 0.9571 - val_loss: 0.1340 - val_accuracy: 0.9625


### **3.2 Generator Dataset**

Khởi tạo từ Generator chúng ta sẽ không phải ghi nhớ toàn bộ dữ liệu vào RAM. Tạo dữ liệu trong quá trình huấn luyện ở mỗi lượt fit từng batch.

Ví dụ khởi tạo data generator để sinh dữ liệu cho mô hình phân loại món ăn theo địa phương.

In [9]:
import pandas as pd

hanoi = ['bún chả hà nội', 'chả cá lã vọng hà nội', 'cháo lòng hà nội', 'ô mai sấu hà nội', 'ô mai', 'chả cá', 'cháo lòng']
hochiminh = ['bánh canh sài gòn', 'hủ tiếu nam vang sài gòn', 'hủ tiếu bò sài gòn', 'banh phở sài gòn', 'bánh phở', 'hủ tiếu']
city = ['hanoi'] * len(hanoi) + ['hochiminh'] * len(hochiminh)
corpus = hanoi+hochiminh

data = pd.DataFrame({'city': city, 'food': corpus})
data.sample(5)

,city,food
9,hochiminh,hủ tiếu bò sài gòn
6,hanoi,cháo lòng
8,hochiminh,hủ tiếu nam vang sài gòn
11,hochiminh,bánh phở
12,hochiminh,hủ tiếu


In [11]:
class Voc(object):
    def __init__(self, corpus):
        self.corpus = corpus
        self.dictionary = {'unk': 0}
        self._initialize_dict(corpus)

    def _add_dict_sentence(self, sentence):
        words = sentence.split(' ')
        for word in words:
            if word not in self.dictionary.keys():
                max_indice = max(self.dictionary.values())
                self.dictionary[word] = (max_indice + 1)

    def _initialize_dict(self, sentences):
        for sentence in sentences:
            self._add_dict_sentence(sentence)

    def _tokenize(self, sentence):
        words = sentence.split(' ')
        token_seq = [self.dictionary[word] for word in words]
        return np.array(token_seq)

voc = Voc(corpus = corpus)

In [12]:
voc.dictionary

{'unk': 0,
 'bún': 1,
 'chả': 2,
 'hà': 3,
 'nội': 4,
 'cá': 5,
 'lã': 6,
 'vọng': 7,
 'cháo': 8,
 'lòng': 9,
 'ô': 10,
 'mai': 11,
 'sấu': 12,
 'bánh': 13,
 'canh': 14,
 'sài': 15,
 'gòn': 16,
 'hủ': 17,
 'tiếu': 18,
 'nam': 19,
 'vang': 20,
 'bò': 21,
 'banh': 22,
 'phở': 23}

Chúng ta sẽ khởi tạo một `random_generator có tác dụng lựa chọn ngẫu nhiên một tên món ăn trong corpus và tokenize chúng.

In [13]:
import tensorflow as tf

cat_indices = {
    'hanoi': 0,
    'hochiminh': 1
}

def generators():
  i = 0
  while True:
    i = np.random.choice(data.shape[0])
    sentence = data.iloc[i, 1]
    x_indice = voc._tokenize(sentence)
    label = data.iloc[i, 0]
    y_indice = cat_indices[label]
    yield x_indice, y_indice
    i += 1

random_generator = tf.data.Dataset.from_generator(
    generators,
    output_types = (tf.float16, tf.float16),
    output_shapes = ((None,), ())
)

random_generator

<FlatMapDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.float16, name=None), TensorSpec(shape=(), dtype=tf.float16, name=None))>

In [15]:
import numpy as np

random_generator_batch = random_generator.shuffle(20).padded_batch(20, padded_shapes=([None], []))
sequence_batch, label = next(iter(random_generator_batch))

print(sequence_batch)
print(label)

tf.Tensor(
[[17. 18.  0.  0.  0.  0.]
 [10. 11. 12.  3.  4.  0.]
 [13. 23.  0.  0.  0.  0.]
 [17. 18.  0.  0.  0.  0.]
 [17. 18. 19. 20. 15. 16.]
 [13. 14. 15. 16.  0.  0.]
 [ 8.  9.  0.  0.  0.  0.]
 [ 1.  2.  3.  4.  0.  0.]
 [10. 11. 12.  3.  4.  0.]
 [17. 18.  0.  0.  0.  0.]
 [ 8.  9.  0.  0.  0.  0.]
 [17. 18. 19. 20. 15. 16.]
 [ 2.  5.  6.  7.  3.  4.]
 [13. 14. 15. 16.  0.  0.]
 [17. 18.  0.  0.  0.  0.]
 [17. 18.  0.  0.  0.  0.]
 [17. 18.  0.  0.  0.  0.]
 [ 8.  9.  3.  4.  0.  0.]
 [ 1.  2.  3.  4.  0.  0.]
 [ 8.  9.  0.  0.  0.  0.]], shape=(20, 6), dtype=float16)
tf.Tensor([1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0.], shape=(20,), dtype=float16)


- Hàm `shuffle(20)` trộn lẫn ngẫu nhiên dữ liệu. Sau đó dữ liệu được chia thành những batch có kích thước là 20.
- Hàm `padded_batch()` padding giá trị 0 sao cho bằng với độ dài của câu dài nhất.

#### **3.2.1 Sử dụng ImageGenerator**

ImageGenerator cũng là một dạng data generator được xây dựng trên framework keras và dành riêng cho dữ liệu ảnh.

Đây là một high level function nên cú pháp đơn giản, rất dễ sử dụng nhưng khả năng tùy biến và can thiệp sâu vào dữ liệu kém.

Khi khởi tạo ImageGenerator chúng ta sẽ khai báo các thủ tục preprocessing image trước khi đưa vào huấn luyện.

In [17]:
image_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True
)

Truyền dữ liệu vào mô hình bằng hàm `flow_from_directory()`

In [20]:
import glob2


#### **3.2.2 Customize ImageGenerator**

Nếu sử dụng các hàm mặc định của image preprocessing trong ImageGenerator thì sẽ giới hạn bởi một số phép biến đổi mà hàm này hỗ trợ, khó can thiệp sâu nên phải Customize lại ImageGenerator

In [21]:
!git clone https://github.com/ardamavi/Dog-Cat-Classifier.git

Cloning into 'Dog-Cat-Classifier'...
Updating files:  65% (1087/1672)
Updating files:  66% (1104/1672)
Updating files:  67% (1121/1672)
Updating files:  68% (1137/1672)
Updating files:  69% (1154/1672)
Updating files:  70% (1171/1672)
Updating files:  71% (1188/1672)
Updating files:  72% (1204/1672)
Updating files:  73% (1221/1672)
Updating files:  74% (1238/1672)
Updating files:  75% (1254/1672)
Updating files:  76% (1271/1672)
Updating files:  77% (1288/1672)
Updating files:  78% (1305/1672)
Updating files:  79% (1321/1672)
Updating files:  80% (1338/1672)
Updating files:  81% (1355/1672)
Updating files:  82% (1372/1672)
Updating files:  83% (1388/1672)
Updating files:  84% (1405/1672)
Updating files:  85% (1422/1672)
Updating files:  86% (1438/1672)
Updating files:  87% (1455/1672)
Updating files:  88% (1472/1672)
Updating files:  89% (1489/1672)
Updating files:  90% (1505/1672)
Updating files:  91% (1522/1672)
Updating files:  92% (1539/1672)
Updating files:  93% (1555/1672)
Updati

In [24]:
ls

 Volume in drive C is New Volume
 Volume Serial Number is 003F-6D10

 Directory of c:\Users\Chi Khang\Documents\Internship\W1

01/28/2023  09:42 AM    <DIR>          .
01/28/2023  09:42 AM    <DIR>          ..
01/28/2023  09:44 AM    <DIR>          Dog-Cat-Classifier
01/28/2023  10:10 AM            27,406 tfdataset.ipynb
01/15/2023  08:03 PM         1,189,164 Xception.ipynb
               2 File(s)      1,216,570 bytes
               3 Dir(s)  74,749,116,416 bytes free


In [36]:
import numpy as np
from tensorflow.keras.utils import Sequence, to_categorical
import cv2

class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self,
                 all_filenames, 
                 labels, 
                 batch_size, 
                 index2class,
                 input_dim,
                 n_channels,
                 n_classes=2, 
                 shuffle=True):
        '''
        all_filenames: list toàn bộ các filename
        labels: nhãn của toàn bộ các file
        batch_size: kích thước của 1 batch
        index2class: index của các class
        input_dim: (width, height) đầu vào của ảnh
        n_channels: số lượng channels của ảnh
        n_classes: số lượng các class 
        shuffle: có shuffle dữ liệu sau mỗi epoch hay không?
        '''
        self.all_filenames = all_filenames
        self.labels = labels
        self.batch_size = batch_size
        self.index2class = index2class
        self.input_dim = input_dim
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        '''
        return:
          Trả về số lượng batch/1 epoch
        '''
        return int(np.floor(len(self.all_filenames) / self.batch_size))

    def __getitem__(self, index):
        '''
        params:
          index: index của batch
        return:
          X, y cho batch thứ index
        '''
        # Lấy ra indexes của batch thứ index
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # List all_filenames trong một batch
        all_filenames_temp = [self.all_filenames[k] for k in indexes]

        # Khởi tạo data
        X, y = self.__data_generation(all_filenames_temp)

        return X, y

    def on_epoch_end(self):
        '''
        Shuffle dữ liệu khi epochs end hoặc start.
        '''
        self.indexes = np.arange(len(self.all_filenames))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, all_filenames_temp):
        '''
        params:
          all_filenames_temp: list các filenames trong 1 batch
        return:
          Trả về giá trị cho một batch.
        '''
        X = np.empty((self.batch_size, *self.input_dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Khởi tạo dữ liệu
        for i, fn in enumerate(all_filenames_temp):
            # Đọc file từ folder name
            img = cv2.imread(fn)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, self.input_dim)
            label = fn.split('/')[3]
            label = self.index2class[label]
    
            X[i,] = img

            # Lưu class
            y[i] = label
        return X, y

In [39]:
import cv2
import glob2

dict_labels = {
    'dog': 0,
    'cat': 1
}

root_folder = 'Dog-Cat-Classifier\\Data\\Train_Data'
fns = glob2.glob(root_folder)
print(len(fns))

image_generator = DataGenerator(
    all_filenames = fns,
    labels = None,
    batch_size = 32,
    index2class = dict_labels,
    input_dim = (224, 224),
    n_channels = 3,
    n_classes = 2,
    shuffle = True
)

1


In [40]:
X, y = image_generator.__getitem__(1)

print(X.shape)
print(y.shape)

(32, 224, 224, 3)
(32,)


IndexError: list index out of range

pwd